In [1]:
import sys, os
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind_from_stats

In [2]:
sys.argv[1] = 'trainingSet.csv'
sys.argv[2] = 'testSet.csv'

trainingDataFilename = sys.argv[1]
testDataFilename = sys.argv[2]
# modelIdx = int(sys.argv[3])
modelIdx = 1

In [3]:
DEPTH_LIMIT = 8
SAMPLE_LIMIT = 50
def class TreeNode(object):
    def __init__(self, class_label, lineage):
        """Node constructor
            @param class_label: class label which this leaf node predicts
            For non-leaf nodes:
                self.val holds the index of feature on which the node splits
            For leaf nodes:
                self.val holds the class label which it predicts
        """
        self.val = class_label
        self.lineage = lineage
        self.left = None
        self.right = None

    def is_leaf(self):
        """Returns: bool: True when node is leaf, False otherwise
        """
        return ((self.left is None) or (self.right is None))

    def expand(self, data, features, N,
               depth=DEFAULT_TREE_DEPTH, downsampling_power=1):
        """Adds 2 children by following splitting criteria
        """
        if not len(data):  # No data given
            self.val = -1
            return
        class_col = [row[-2] for row in data]
        class_label = int(class_col.count(1) > class_col.count(0))
        eligible_features = [f for f in features if f not in self.lineage]
        available_features = \
            sample(eligible_features,
                   int(round(len(eligible_features)**downsampling_power)))
        if len(self.lineage) <= depth \
           and (sum([row[-3] for row in data]) * N) > 9 \
           and len(available_features) > 0:  # eligible non-leaf node
            ginis = {i: gini(data, i) for i in available_features}
            split_idx = min(ginis, key=ginis.get)
            left_data = [row for row in data if row[split_idx] == 0]
            right_data = [row for row in data if row[split_idx] == 1]
            if sum([r[-3] for r in left_data]) and \
               sum([r[-3] for r in right_data]):  # Confirmed non-leaf node
                self.val = split_idx
                self.left = TreeNode(class_label, self.lineage + [split_idx])
                self.left.expand(left_data, features, N, depth=depth,
                                 downsampling_power=downsampling_power)
                self.right = TreeNode(class_label, self.lineage + [split_idx])
                self.right.expand(right_data, features, N, depth=depth,
                                  downsampling_power=downsampling_power)
            else:
                self.val = class_label
        else:
            self.val = class_label
def gini(data, attr):
    gini = 0.0
    total = len(data)
    pos = 1.0 * data[attr].value_counts()[0]/total
    neg = 1.0 * data[attr].value_counts()[1]/total

    return 1 - pos ** 2 - neg ** 2

In [4]:
def decisionTree(trainingSet, testSet):
    print 'decisionTree'
    print gini(trainingSet, 'decision')

In [5]:
def bagging(trainingSet, testSet):
    print 'bagging'

In [6]:
def randomForests(trainingSet, testSet):
    print 'randomForests'

In [8]:
trainingSet = pd.read_csv(trainingDataFilename)
testSet = pd.read_csv(testDataFilename)

keys = trainingSet.keys()
keys = keys.drop('decision')

if modelIdx == 1:
#     for 
    decisionTree(trainingSet, testSet)
elif modelIdx == 2:
    bagging(trainingSet, testSet)
elif modelIdx == 3:
    randomForests(trainingSet, testSet)
else:
    print 'modelIdx error'

49
decisionTree
0.4921875
